In [12]:
from Acesso import Login
from Query import Query
from Tempo import DataHora
from datetime import timedelta
import pandas as pd

data=DataHora()

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

In [2]:
querys={
    
    'Cliente':
    
    """
    
    SELECT [ID Cliente],[Razão Social],[Nome Fantasia],[Status do Cliente],[Principal],[Limite de Crédito],
    [Última Compra],
    DATEADD(day,-90,[Última Compra]) AS 'Data Inicial'
    FROM netfeira.vw_cliente
    WHERE [Dias Compra]<366
    
    """,
    
    'TargetEstatistico':
    
    """
    
    SELECT * FROM netfeira.vw_targetestatistico
    
    """
    
}

In [3]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [4]:
tabelas_df['Cliente'].columns

Index(['ID Cliente', 'Razão Social', 'Nome Fantasia', 'Status do Cliente',
       'Principal', 'Limite de Crédito', 'Última Compra', 'Data Inicial'],
      dtype='object')

In [5]:
tabelas_df['TargetEstatistico'].columns

Index(['Origem', 'ID Empresa', 'ID Cliente', 'ID Vendedor', 'Pedido', 'Nfe',
       'Tipo de Pedido', 'Tipo de Operação', 'ID Cadastro', 'Data de Emissão',
       'Data de Faturamento', 'SKU', 'Seq', 'Qtde', 'Unid. VDA', 'Fator VDA',
       'Preço Tabela', 'Desc %', 'Desc R$', 'Valor VDA', 'Preço Unitário',
       'Total Geral', 'Custo CUE', 'Margem Bruta R$', 'Custo Capado CUE',
       'Total AV', 'PIS R$', 'COFINS R$', 'ICMS R$', 'Verba R$',
       'Margem CTB R$', 'Situação', 'Total Geral AV'],
      dtype='object')

In [6]:
tabelas_df['TargetEstatistico']=tabelas_df['TargetEstatistico'].loc[(tabelas_df['TargetEstatistico']['Tipo de Operação']=='VENDAS')&(tabelas_df['TargetEstatistico']['Situação']=='FATURADO')]

In [7]:
tabelas_df['Cliente']['Total']=tabelas_df['Cliente'].apply(lambda info: tabelas_df['TargetEstatistico']['Total AV'].loc[(tabelas_df['TargetEstatistico']['Data de Faturamento'].between(info['Data Inicial'],info['Última Compra']))&(tabelas_df['TargetEstatistico']['ID Cliente']==info['ID Cliente'])].sum(),axis=1)

In [8]:
tabelas_df['Cliente']['Pedidos']=tabelas_df['Cliente'].apply(lambda info: len(tabelas_df['TargetEstatistico']['Total AV'].loc[(tabelas_df['TargetEstatistico']['Data de Faturamento'].between(info['Data Inicial'],info['Última Compra']))&(tabelas_df['TargetEstatistico']['ID Cliente']==info['ID Cliente'])].unique().tolist()),axis=1)

In [9]:
tabelas_df['Cliente']['Média R$']=tabelas_df['Cliente'].apply(lambda info: round(info['Total']/info['Pedidos'],2) if info['Pedidos']!=0 else 0,axis=1)

In [15]:
tabelas_df['Cliente']=tabelas_df['Cliente'].loc[(~tabelas_df['Cliente']['Razão Social'].str.contains('FUNC'))]

In [19]:
tabelas_df['Cliente'].loc[tabelas_df['Cliente']['Média R$']==0,'Média R$']=100

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [20]:
tabelas_df['Cliente']

,ID Cliente,Razão Social,Nome Fantasia,Status do Cliente,Principal,Limite de Crédito,Última Compra,Data Inicial,Total,Pedidos,Média R$
0,10,DE MARCHI IND. E COM. DE FRUTAS LTDA,DE MARCHI,ATIVO,AVULSO,NaN,2022-05-26,2022-02-25,518735.08,5,103747.02
1,989,MCCAIN DO BRASIL ALIMENTOS LTDA.,MCCAIN DO BRASIL,ATIVO,AVULSO,NaN,2022-06-14,2022-03-16,0.00,0,100.00
2,1578,CAFE E PAO DE QUEIJO LARGO 13 DE MAIO -,CAFE E PAO DE QUEIJO,ATIVO,RENATAAP,500.0,2022-06-09,2022-03-11,826.20,1,826.20
3,1593,COMERCIO VAREJISTA DE HORTIFRUTIGRANJ VI,SACOLAO VILA SONIA,ATIVO,RODRIGOA,NaN,2022-07-23,2022-04-24,19685.20,40,492.13
4,1658,RITA DE CASSIA CORREA PACHECO,SACOLAO DA ECONOMIA,ATIVO,RODRIGOA,NaN,2022-06-30,2022-04-01,10291.85,67,153.61
...,...,...,...,...,...,...,...,...,...,...,...
5866,28270,JOTA ''A'' SALES LTDA,MERCADINHO JOTA A,ATIVO,CLAUDIOP,200.0,2022-07-25,2022-04-26,0.00,0,100.00
5867,28271,DOCURA GELADA SORVETES LTDA,ICE CREAM ROLL SHOPP,ATIVO,MAYRAQS,200.0,2022-07-25,2022-04-26,184.62,1,184.62
5868,28272,IVANILDA ETELVINA DE CARVALHO XAVIER 30582574811,OAKS FOOD,ATIVO,MARCOSLI,250.0,2022-07-25,2022-04-26,0.00,0,100.00
5869,28273,RENATA MARIA DO NASCIMENTO,RENATA MARIA DO NASC,ATIVO,MAYRAQS,150.0,2022-07-25,2022-04-26,0.00,0,100.00


In [14]:
tabelas_df['Cliente'].to_excel('Clientes.xlsx',index=False,encoding='UTF-8')

NameError: name 'tabelas_df' is not defined

In [16]:
from glob import glob
import pandas as pd

anexo=glob('*.xlsx')

In [17]:
credito_df=pd.read_excel(anexo[0])

In [18]:
credito_df

,ID Cliente,Razão Social,Nome Fantasia,Status do Cliente,Principal,Limite de Crédito,Última Compra,Data Inicial,Total,Pedidos,Média R$
0,10,DE MARCHI IND. E COM. DE FRUTAS LTDA,DE MARCHI,ATIVO,AVULSO,NaN,2022-05-26,2022-02-25,518735.08,5,103747.02
1,989,MCCAIN DO BRASIL ALIMENTOS LTDA.,MCCAIN DO BRASIL,ATIVO,AVULSO,NaN,2022-06-14,2022-03-16,0.00,0,100.00
2,1593,COMERCIO VAREJISTA DE HORTIFRUTIGRANJ VI,SACOLAO VILA SONIA,ATIVO,RODRIGOA,NaN,2022-07-23,2022-04-24,19685.20,40,492.13
3,1658,RITA DE CASSIA CORREA PACHECO,SACOLAO DA ECONOMIA,ATIVO,RODRIGOA,NaN,2022-06-30,2022-04-01,10291.85,67,153.61
4,1671,LAPA COMERCIAL HORTIFRUTIGRANJEIROS LTDA,SACOLAO DA ECONOMIA,ATIVO,RODRIGOA,NaN,2022-07-21,2022-04-22,17658.54,33,535.11
...,...,...,...,...,...,...,...,...,...,...,...
3293,28233,MERCADO E ROTISSERIE TOCHA LTDA,MERCADO TOCHA 03,ATIVO,ADEMIRFE,NaN,2022-07-19,2022-04-20,266.45,4,66.61
3294,28243,XINQI MERCADO LTDA,MERCADO XINQI,ATIVO,CLAUDIOP,NaN,2022-07-22,2022-04-23,953.17,4,238.29
3295,28250,CLAUDEMIRDO CAMIHAO,CLAUDEMIR CAMIAO,ATIVO,LOJA,NaN,2022-07-21,2022-04-22,11.03,1,11.03
3296,28259,REDE D'OR SAO LUIZ S.A.,MATERNIDADE SAO LUIZ,ATIVO,CRIS,NaN,2022-07-22,2022-04-23,200.00,1,200.00


In [13]:
for indice,linha in credito_df.iterrows():
    
    query=f"""
    
    UPDATE cliente
    SET vl_lim_cred={linha['Média R$']},
    VerificaSomenteLimiteCliente=1,
    VerificaSomenteLimiteColigacao=1,
    VerificaSomenteLimiteGrupo=1
    WHERE cd_clien={linha['ID Cliente']}
    
    """
    
    #sql.Salvar(query,sql.conectando)
    
    pass

print('Alterado com sucesso!')

Alterado com sucesso!
